<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/homeexercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding, AutoTokenizer

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }



# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [ ]:
!pip install optuna

  Using cached optuna-4.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.6-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.1.0-py3-none-any.whl (364 kB)
Using cached alembic-1.14.0-py3-none-any.whl (233 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached Mako-1.3.6-py3-none-any.whl (78 kB)


In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer", # where to put everything that is trained
    per_device_train_batch_size=8,  # amount of sentences model looks at before anything is updated
    per_device_eval_batch_size=8,
    logging_steps=8,                # how many times results are logged/track/output
    num_train_epochs=10,
    eval_strategy="epoch",          # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=1e-5,             # influences how weight changes in function, determines how big a step you take in valley
    weight_decay=0.005,              # sum of weights added to loss function, regulization method, penalizes large weights
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.682200,0.693174,0.468750
2,0.684100,0.688901,0.468750
3,0.663700,0.672419,0.500000
4,0.612400,0.644293,0.781250
5,0.562400,0.607863,0.812500
6,0.462400,0.553763,0.843750
7,0.375600,0.513987,0.812500
8,0.314200,0.478871,0.875000
9,0.267300,0.463749,0.875000
10,0.272500,0.459768,0.843750


TrainOutput(global_step=160, training_loss=0.5006450787186623, metrics={'train_runtime': 1239.8815, 'train_samples_per_second': 1.032, 'train_steps_per_second': 0.129, 'total_flos': 36337463293824.0, 'train_loss': 0.5006450787186623, 'epoch': 10.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[ 0.6254548 , -0.24747477],
       [-1.1510874 ,  0.90899223],
       [ 0.24678981,  0.26977423],
       [-0.71183646,  0.71441317],
       [-0.46013474,  0.5440769 ],
       [-0.96429515,  0.80689865],
       [ 0.5574993 , -0.399373  ],
       [ 0.7924975 , -0.6476236 ],
       [ 0.02069771,  0.24121338],
       [ 0.11433366, -0.02530708],
       [-0.1796627 ,  0.63809395],
       [ 0.6276436 , -0.4834797 ],
       [ 0.13189234, -0.30014634],
       [-0.7639835 ,  0.64128625],
       [-0.48282862,  0.6144321 ],
       [-0.32334077,  0.38191673],
       [ 0.00915378,  0.10162576],
       [ 0.14100838,  0.28745154],
       [-0.2988384 ,  0.41972435],
       [ 0.77839196, -0.5631106 ],
       [ 0.07537009,  0.380215  ],
       [ 0.5557085 , -0.31079784],
       [ 0.556118  , -0.1604549 ],
       [ 0.7119608 , -0.48887908],
       [-0.8648093 ,  0.8594225 ],
       [ 0.95808923, -0.7401277 ],
       [ 0.30001524, -0.08070067],
       [-0.05270683,  0.47

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [ ]:
import torch

test_str = "I hate this movie!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/checkpoint-144")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

NEGATIVE


In [ ]:
test_dataset = small_imdb_dataset['test']

test_results = trainer.predict(small_tokenized_dataset['test'])
print("Test set evaluation results:", test_results)

Test set evaluation results: PredictionOutput(predictions=array([[ 0.87845576, -0.61240655],
       [ 0.8294658 , -0.6899486 ],
       [ 0.46415454, -0.18060079],
       [-0.22983849,  0.15763688],
       [ 0.614255  , -0.62235266],
       [-0.80942875,  0.89223933],
       [-0.6827831 ,  0.68266195],
       [ 0.584398  , -0.16275622],
       [-0.62700206,  0.7141466 ],
       [ 0.13370642,  0.24209908],
       [ 0.31586063,  0.12510666],
       [-0.8778178 ,  0.78590673],
       [ 0.51914144, -0.02581131],
       [ 0.4363265 , -0.20913824],
       [-0.74065256,  0.73400635],
       [-0.56064963,  0.5335861 ],
       [-0.4799466 ,  0.6638038 ],
       [-0.9617985 ,  0.7026859 ],
       [ 0.86578906, -0.51373667],
       [-0.36208847,  0.47121   ],
       [ 0.13761687,  0.05034046],
       [ 0.04363983,  0.24811244],
       [ 1.0411247 , -0.95407206],
       [ 0.65384215, -0.3879072 ],
       [-0.10314952,  0.10691467],
       [-0.3827263 ,  0.56766206],
       [ 0.7845536 , -0.5548713 

In [ ]:
tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)

# new trainer for evaluation
trainer = Trainer(
    model=fine_tuned_model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['test'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(test_results)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

{'eval_loss': 0.5175219774246216, 'eval_model_preparation_time': 0.0032, 'eval_accuracy': 0.84375, 'eval_runtime': 10.1167, 'eval_samples_per_second': 3.163, 'eval_steps_per_second': 0.395}
